In [1]:
# Create 'salary_data_converted' file

# Import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Load the CSV file into a DataFrame
df = pd.read_csv(r"C:\Users\Lenovo\Downloads\MAYSQL\Cleaned_survey_data.csv") #your_file_path

# Select specific columns from the DataFrame
selected_columns = ['Age', 'Annual_salary', 'Currency', 'Country', 'professional_work_experience_overall', 'professional_work_experience_field',
'education_level', 'Gender', 'grouped_industry']

df1 = df[selected_columns]

# List of URLs for currency conversion rates
urls = [
    'https://www.google.com/search?q=eur+to+usd&oq=eur+&gs_lcrp=EgZjaHJvbWUqDAgBEAAYQxiABBiKBTIGCAAQRRg5MgwIARAAGEMYgAQYigUyDAgCEAAYQxiABBiKBTIMCAMQABhDGIAEGIoFMgwIBBAAGEMYgAQYigUyDAgFEAAYQxiABBiKBTIMCAYQABhDGIAEGIoFMgwIBxAAGEMYgAQYigUyDAgIEAAYQxiABBiKBTIMCAkQABhDGIAEGIoF0gEJNTMwNmowajE1qAIIsAIB&sourceid=chrome&ie=UTF-8', 
    'https://www.google.com/search?q=cad+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIKtH9xIPQzKU85gpxDLsMM57VhyzA%3A1721901733125&ei=pSKiZsSjB9niwPAP7ZmP-Q8&oq=CAD+tu&gs_lp=Egxnd3Mtd2l6LXNlcnAiBkNBRCB0dSoCCAEyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGApIyiNQgQlYzBZwAXgBkAEAmAH-AaAB_QaqAQUwLjEuM7gBAcgBAPgBAfgBApgCBaACkAeoAhTCAgcQIxgnGOoCwgITEAAYgAQYQxi0AhiKBRjqAtgBAcICChAjGIAEGCcYigXCAgoQABiABBhDGIoFwgIFEAAYgASYAwq6BgYIARABGAGSBwUxLjEuM6AHrBc&sclient=gws-wiz-serp', 
    'https://www.google.com/search?q=gbp+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIIQdbjCM1aR0iFaG-zrZHJGaTnwDg%3A1721911959677&ei=l0qiZu3-KMSTwPAPsbC-yQ0&oq=GBP+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCkdCUCB0byB1c2QqAggAMgoQABiABBhDGIoFMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgUQABiABDIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5I1URQgzNY1zZwBHgBkAEAmAH8AaABzgOqAQMyLTK4AQHIAQD4AQH4AQKYAgagAuIDwgIKEAAYsAMY1gQYR8ICDRAAGIAEGLADGEMYigWYAwCIBgGQBgqSBwU0LjAuMqAHhQk&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=AUD+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWILcH6_iVwyEauW0SLrwYSRrY1xM4g%3A1721912318665&ei=_kuiZrSlKLCswPAPhZ-liQI&ved=0ahUKEwi0nNqgn8KHAxUwFhAIHYVPKSEQ4dUDCBA&uact=5&oq=AUD+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCkFVRCB0byB1c2QyChAAGIAEGEMYigUyBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5IuB1Q1QZY0xVwAXgBkAEAmAGgAqABswqqAQUwLjEuNbgBA8gBAPgBAZgCBKAC1wXCAgoQABiwAxjWBBhHwgINEAAYgAQYsAMYQxiKBZgDAIgGAZAGCpIHBTEuMS4yoAf9Hw&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=ZAR+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIKw688_i0Rx0MuHCPak0K4X8PC6Zg%3A1721912391525&ei=R0yiZsDbH-uA1fIP0simmAo&ved=0ahUKEwiAm7nDn8KHAxVrQFUIHVKkCaMQ4dUDCBA&uact=5&oq=ZAR+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiClpBUiB0byB1c2QyChAAGIAEGEMYigUyBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5IwhtQAFiHF3AAeACQAQCYAcYCoAHdB6oBBTItMy4xuAEDyAEA-AEBmAIEoALkB5gDAJIHBTItMy4xoAeHFA&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=jpy+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIIOx_DWuRHY61pmhW0Ec9fSPdsmgQ%3A1721912453385&ei=hUyiZvyXF4KA1fIPz7SGiQM&ved=0ahUKEwi87vjgn8KHAxUCQFUIHU-aITEQ4dUDCBA&uact=5&oq=jpy+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCmpweSB0byB1c2QyDxAAGIAEGEMYigUYRhiCAjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIbEAAYgAQYQxiKBRhGGIICGJcFGIwFGN0E2AEBSJ8SUABYuA5wAHgBkAEAmAGMAqABgQeqAQUwLjEuM7gBA8gBAPgBAZgCBKACkQeYAwC6BgYIARABGBOSBwUwLjEuM6AHpRY&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=CHF+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIL_vMk9lRbEgRMqkAEbuB3lPUCanQ%3A1721912494923&ei=rkyiZquMOMG5wPAP18S90QQ&ved=0ahUKEwirm-D0n8KHAxXBHBAIHVdiL0oQ4dUDCBA&oq=CHF+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCkNIRiB0byB1c2QyChAAGIAEGEMYigUyBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5IxhtQmQZYiw1wAXgBkAEAmAGSAqAB7gaqAQUwLjEuM7gBDMgBAPgBAfgBApgCA6ACkwTCAgoQABiwAxjWBBhHwgINEAAYgAQYsAMYQxiKBZgDAIgGAZAGCpIHBTEuMC4yoAfZFA&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=SEK+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIIipmr_6wrt_GFixqPAzdcrdB9bag%3A1721912575665&ei=_0yiZr-fKOu-wPAP5uWD8AU&ved=0ahUKEwj_mqCboMKHAxVrHxAIHebyAF4Q4dUDCBA&uact=5&oq=SEK+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiClNFSyB0byB1c2QyChAAGIAEGEMYigUyBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5I5AxQ1AVY1AVwAXgBkAEAmAHHAaABxwGqAQMwLjG4AQPIAQD4AQL4AQGYAgKgAs8BwgIKEAAYsAMY1gQYR8ICDRAAGIAEGLADGEMYigWYAwCIBgGQBgqSBwUxLjAuMaAHzgQ&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=HKD+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIJawGOmkpE4jbFLdsdCmqDnnm94Lg%3A1721912641808&ei=QU2iZpuGMb_UwPAPgYSCqQQ&ved=0ahUKEwjbquW6oMKHAxU_KhAIHQGCIEUQ4dUDCBA&oq=HKD+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCkhLRCB0byB1c2QyCxAAGIAEGJECGIoFMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeSKsSULoGWLoGcAF4AZABAJgBzAGgAcwBqgEDMi0xuAEMyAEA-AEC-AEBmAICoALUAcICChAAGLADGNYEGEfCAg0QABiABBiwAxhDGIoFmAMAiAYBkAYKkgcFMS4wLjGgB84E&sclient=gws-wiz-serp'
]

# List of currency codes corresponding to the URLs
currency_codes = ['EUR', 'CAD', 'GBP', 'AUD/NZD', 'ZAR', 'JPY', 'CHF', 'SEK', 'HKD']

# Dictionary to store the retrieved exchange rates
exchange_rates = {}

# Headers to be used for web requests
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", 
    "Accept-Encoding": "gzip, deflate", 
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
    "DNT": "1", 
    "Connection": "close", 
    "Upgrade-Insecure-Requests": "1"
}

# Fetch exchange rates from the URLs
for url, currency in zip(urls, currency_codes):
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    span = soup.find('span', class_="DFlfde SwHCTb")
    if span:
        value_span = span.get_text(strip=True)
        value_span = value_span.replace(',', '.')
        value = float(value_span)
        exchange_rates[currency] = value
    else:
        print(f'Span not found in URL: {url}')

# Print the retrieved exchange rates
print("Exchange rates retrieved:")
print(exchange_rates)

# Function to convert salaries to USD based on the exchange rates
def convert_to_usd(row):
    currency = row['Currency']
    salary = row['Annual_salary']
    rate = exchange_rates.get(currency, 1)
    return salary * rate

# Apply the conversion function and add a new column for salaries in USD
df1.loc[:, 'salary_in_usd'] = df1.apply(convert_to_usd, axis=1)

# Save the updated DataFrame to a new CSV file
df1.to_csv('salary_data_converted.csv', index=False)

# Notify that the conversion is complete and the file is saved
print("Conversion completed. New file saved as 'salary_data_converted.csv'.")

Exchange rates retrieved:
{'EUR': 1.09, 'CAD': 0.72, 'GBP': 1.29, 'AUD/NZD': 0.66, 'ZAR': 0.055, 'JPY': 0.0065, 'CHF': 1.13, 'SEK': 0.093, 'HKD': 0.13}
Conversion completed. New file saved as 'salary_data_converted.csv'.


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1172\4179556806.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, 'salary_in_usd'] = df1.apply(convert_to_usd, axis=1)


In [36]:
# Update 'salary_data_converted' file

import pandas as pd
import requests
from bs4 import BeautifulSoup

df2 = pd.read_csv(r"C:\Users\Lenovo\salary_data_converted.csv") #your_file_path

urls = [
    'https://www.google.com/search?q=eur+to+usd&oq=eur+&gs_lcrp=EgZjaHJvbWUqDAgBEAAYQxiABBiKBTIGCAAQRRg5MgwIARAAGEMYgAQYigUyDAgCEAAYQxiABBiKBTIMCAMQABhDGIAEGIoFMgwIBBAAGEMYgAQYigUyDAgFEAAYQxiABBiKBTIMCAYQABhDGIAEGIoFMgwIBxAAGEMYgAQYigUyDAgIEAAYQxiABBiKBTIMCAkQABhDGIAEGIoF0gEJNTMwNmowajE1qAIIsAIB&sourceid=chrome&ie=UTF-8', 
    'https://www.google.com/search?q=cad+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIKtH9xIPQzKU85gpxDLsMM57VhyzA%3A1721901733125&ei=pSKiZsSjB9niwPAP7ZmP-Q8&oq=CAD+tu&gs_lp=Egxnd3Mtd2l6LXNlcnAiBkNBRCB0dSoCCAEyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGAoyBxAAGIAEGApIyiNQgQlYzBZwAXgBkAEAmAH-AaAB_QaqAQUwLjEuM7gBAcgBAPgBAfgBApgCBaACkAeoAhTCAgcQIxgnGOoCwgITEAAYgAQYQxi0AhiKBRjqAtgBAcICChAjGIAEGCcYigXCAgoQABiABBhDGIoFwgIFEAAYgASYAwq6BgYIARABGAGSBwUxLjEuM6AHrBc&sclient=gws-wiz-serp', 
    'https://www.google.com/search?q=gbp+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIIQdbjCM1aR0iFaG-zrZHJGaTnwDg%3A1721911959677&ei=l0qiZu3-KMSTwPAPsbC-yQ0&oq=GBP+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCkdCUCB0byB1c2QqAggAMgoQABiABBhDGIoFMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgUQABiABDIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5I1URQgzNY1zZwBHgBkAEAmAH8AaABzgOqAQMyLTK4AQHIAQD4AQH4AQKYAgagAuIDwgIKEAAYsAMY1gQYR8ICDRAAGIAEGLADGEMYigWYAwCIBgGQBgqSBwU0LjAuMqAHhQk&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=AUD+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWILcH6_iVwyEauW0SLrwYSRrY1xM4g%3A1721912318665&ei=_kuiZrSlKLCswPAPhZ-liQI&ved=0ahUKEwi0nNqgn8KHAxUwFhAIHYVPKSEQ4dUDCBA&uact=5&oq=AUD+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCkFVRCB0byB1c2QyChAAGIAEGEMYigUyBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5IuB1Q1QZY0xVwAXgBkAEAmAGgAqABswqqAQUwLjEuNbgBA8gBAPgBAZgCBKAC1wXCAgoQABiwAxjWBBhHwgINEAAYgAQYsAMYQxiKBZgDAIgGAZAGCpIHBTEuMS4yoAf9Hw&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=ZAR+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIKw688_i0Rx0MuHCPak0K4X8PC6Zg%3A1721912391525&ei=R0yiZsDbH-uA1fIP0simmAo&ved=0ahUKEwiAm7nDn8KHAxVrQFUIHVKkCaMQ4dUDCBA&uact=5&oq=ZAR+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiClpBUiB0byB1c2QyChAAGIAEGEMYigUyBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5IwhtQAFiHF3AAeACQAQCYAcYCoAHdB6oBBTItMy4xuAEDyAEA-AEBmAIEoALkB5gDAJIHBTItMy4xoAeHFA&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=jpy+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIIOx_DWuRHY61pmhW0Ec9fSPdsmgQ%3A1721912453385&ei=hUyiZvyXF4KA1fIPz7SGiQM&ved=0ahUKEwi87vjgn8KHAxUCQFUIHU-aITEQ4dUDCBA&uact=5&oq=jpy+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCmpweSB0byB1c2QyDxAAGIAEGEMYigUYRhiCAjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIbEAAYgAQYQxiKBRhGGIICGJcFGIwFGN0E2AEBSJ8SUABYuA5wAHgBkAEAmAGMAqABgQeqAQUwLjEuM7gBA8gBAPgBAZgCBKACkQeYAwC6BgYIARABGBOSBwUwLjEuM6AHpRY&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=CHF+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIL_vMk9lRbEgRMqkAEbuB3lPUCanQ%3A1721912494923&ei=rkyiZquMOMG5wPAP18S90QQ&ved=0ahUKEwirm-D0n8KHAxXBHBAIHVdiL0oQ4dUDCBA&oq=CHF+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCkNIRiB0byB1c2QyChAAGIAEGEMYigUyBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5IxhtQmQZYiw1wAXgBkAEAmAGSAqAB7gaqAQUwLjEuM7gBDMgBAPgBAfgBApgCA6ACkwTCAgoQABiwAxjWBBhHwgINEAAYgAQYsAMYQxiKBZgDAIgGAZAGCpIHBTEuMC4yoAfZFA&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=SEK+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIIipmr_6wrt_GFixqPAzdcrdB9bag%3A1721912575665&ei=_0yiZr-fKOu-wPAP5uWD8AU&ved=0ahUKEwj_mqCboMKHAxVrHxAIHebyAF4Q4dUDCBA&uact=5&oq=SEK+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiClNFSyB0byB1c2QyChAAGIAEGEMYigUyBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB5I5AxQ1AVY1AVwAXgBkAEAmAHHAaABxwGqAQMwLjG4AQPIAQD4AQL4AQGYAgKgAs8BwgIKEAAYsAMY1gQYR8ICDRAAGIAEGLADGEMYigWYAwCIBgGQBgqSBwUxLjAuMaAHzgQ&sclient=gws-wiz-serp',
    'https://www.google.com/search?q=HKD+to+usd&sca_esv=8dca5a6e4fb2a16a&sca_upv=1&sxsrf=ADLYWIJawGOmkpE4jbFLdsdCmqDnnm94Lg%3A1721912641808&ei=QU2iZpuGMb_UwPAPgYSCqQQ&ved=0ahUKEwjbquW6oMKHAxU_KhAIHQGCIEUQ4dUDCBA&oq=HKD+to+usd&gs_lp=Egxnd3Mtd2l6LXNlcnAiCkhLRCB0byB1c2QyCxAAGIAEGJECGIoFMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeSKsSULoGWLoGcAF4AZABAJgBzAGgAcwBqgEDMi0xuAEMyAEA-AEC-AEBmAICoALUAcICChAAGLADGNYEGEfCAg0QABiABBiwAxhDGIoFmAMAiAYBkAYKkgcFMS4wLjGgB84E&sclient=gws-wiz-serp'
]

currency_codes = ['EUR', 'CAD', 'GBP', 'AUD/NZD', 'ZAR', 'JPY', 'CHF', 'SEK', 'HKD']

exchange_rates = {}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", 
    "Accept-Encoding": "gzip, deflate", 
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
    "DNT": "1", 
    "Connection": "close", 
    "Upgrade-Insecure-Requests": "1"
}

for url, currency in zip(urls, currency_codes):
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    span = soup.find('span', class_="DFlfde SwHCTb")
    if span:
        value_span = span.get_text(strip=True)
        value_span = value_span.replace(',', '.')
        value = float(value_span)
        exchange_rates[currency] = value
    else:
        print(f'Span not found in URL: {url}')

print("Exchange rates retrieved:")
print(exchange_rates)

def convert_to_usd(row):
    currency = row['Currency']
    salary = row['Annual_salary']
    rate = exchange_rates.get(currency, 1)
    return salary * rate

df2.loc[:, 'salary_in_usd'] = df2.apply(convert_to_usd, axis=1)

df2.to_csv(r"C:\Users\Lenovo\salary_data_converted.csv", index=False) #your_file_path

print("Conversion completed. File updated.")


Exchange rates retrieved:
{'EUR': 1.09, 'CAD': 0.72, 'GBP': 1.29, 'AUD/NZD': 0.66, 'ZAR': 0.055, 'JPY': 0.0065, 'CHF': 1.13, 'SEK': 0.092, 'HKD': 0.13}
Conversion completed. File updated.
